<a href="https://colab.research.google.com/github/gyhou/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference) **water-project-cost-sharing**

- **immigration**: Not enough certainty level. p-value = 0.083

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel, ttest_1samp
# ttest_ind - 2 function T test
# ttest_1samp - 1 function(sample) T test

In [122]:
### YOUR CODE STARTS HERE

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', header=None)
print(df.shape)
df.head()

(435, 17)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.names

In [127]:
df.columns = ["party", "handicapped-infants", "water-project-cost-sharing"
              , "adoption-of-the-budget-resolution", "physician-fee-freeze"
              , "el-salvador-aid", "religious-groups-in-schools"
              , "anti-satellite-test-ban", "aid-to-nicaraguan-contras"
              , "mx-missile", "immigration", "synfuels-corporation-cutback"
              , "education-spending", "superfund-right-to-sue", "crime"
              , "duty-free-exports", "export-administration-act-south-africa"
             ]
df.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [130]:
# df[df == 'n'] = 0
# df[df == 'y'] = 1
# df[df == '?'] = np.nan
# df = df.apply(pd.to_numeric, downcast='float', errors='ignore')
# convert to float
# 2 sample test (ttest_ind) needs to be float
df = df.replace({'n':0.0,'y':1.0,'?':np.nan})
df.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
rep = df[df['party']=='republican']
dem = df[df['party']=='democrat']

In [134]:
ttest_ind(dem['physician-fee-freeze'], rep['physician-fee-freeze'], nan_policy='omit')

Ttest_indResult(statistic=-49.36708157301406, pvalue=1.994262314074344e-177)

In [135]:
for col in range(len(df.columns)-1):
  issue = df.columns[col+1]
  print('Voting Issue: ' + issue)
  
  t, p = ttest_ind(dem[issue], rep[issue], nan_policy='omit')
  
  if p < 0.01 and dem[issue].mean() > rep[issue].mean():
    print('Significant Democrat Support: p < 0.01')
  elif p < 0.01 and dem[issue].mean() < rep[issue].mean():
    print('Significant Republican Support: p < 0.01')
  elif p > 0.1:
    print('There may not be much of a difference: p > 0.1')
    print('p-value = %0.3f' % p)
  else:
    print('Not enough certainty level.')
    print('p-value = %0.3f' % p)
  
  d_per = dem[issue].mean()*100
  r_per = rep[issue].mean()*100
  print("Democrat Support: %0.2f%%" % d_per)
  print("Republican Support: %0.2f%%" % r_per)
  print()

Voting Issue: handicapped-infants
Significant Democrat Support: p < 0.01
Democrat Support: 60.47%
Republican Support: 18.79%

Voting Issue: water-project-cost-sharing
There may not be much of a difference: p > 0.1
p-value = 0.929
Democrat Support: 50.21%
Republican Support: 50.68%

Voting Issue: adoption-of-the-budget-resolution
Significant Democrat Support: p < 0.01
Democrat Support: 88.85%
Republican Support: 13.41%

Voting Issue: physician-fee-freeze
Significant Republican Support: p < 0.01
Democrat Support: 5.41%
Republican Support: 98.79%

Voting Issue: el-salvador-aid
Significant Republican Support: p < 0.01
Democrat Support: 21.57%
Republican Support: 95.15%

Voting Issue: religious-groups-in-schools
Significant Republican Support: p < 0.01
Democrat Support: 47.67%
Republican Support: 89.76%

Voting Issue: anti-satellite-test-ban
Significant Democrat Support: p < 0.01
Democrat Support: 77.22%
Republican Support: 24.07%

Voting Issue: aid-to-nicaraguan-contras
Significant Democra